## HW2

In [6]:
import numpy as np
from sklearn.preprocessing import StandardScaler

# housing dataset:
train_house = np.loadtxt("housing_data/train.txt")
test_house = np.loadtxt("housing_data/test.txt")

Q1 A - Housing dataset with Linear Reg (normal eqns)

In [15]:
# 1. separate features and labels
X_train = train_house[:, :-1]
y_train = train_house[:, -1]
X_test = test_house[:, :-1]
y_test = test_house[:, -1]

# 2. Normalization step
scaler = StandardScaler()
scaler.fit(X_train)
X_train_normalized = scaler.transform(X_train)
X_test_normalized = scaler.transform(X_test)

# 3. bias step
ones_col_Xtrain = np.ones((X_train_normalized.shape[0], 1))
ones_col_Xtest = np.ones((X_test_normalized.shape[0], 1))

Xtrain_house_bias = np.hstack([ones_col_Xtrain, X_train_normalized])
Xtest_house_bias = np.hstack([ones_col_Xtest, X_test_normalized])

# 4. Normal Eqn step
Xtrain_T = Xtrain_house_bias.T
Xtrain_dot = np.dot(Xtrain_T, Xtrain_house_bias)
Xtrain_inv = np.linalg.inv(Xtrain_dot)
Xtrain_y_dot = np.dot(Xtrain_T, y_train)
theta_1 = np.dot(Xtrain_inv, Xtrain_y_dot)

# 5. Prediction step
train_house_pred = np.dot(Xtrain_house_bias, theta_1)
test_house_pred = np.dot(Xtest_house_bias, theta_1)

# 6. MSE for linear reg with normal equations
train_mse_1 = np.mean((train_house_pred-y_train)**2)
test_mse_1 = np.mean((test_house_pred-y_test)**2)

print(f"TRAIN-MSE-1 Housing data with Linear Reg: {train_mse_1}")
print(f"TEST-MSE-1 Housing data with Linear Reg: {test_mse_1}")

TRAIN-MSE-1 Housing data with Linear Reg: 22.081273187013167
TEST-MSE-1 Housing data with Linear Reg: 22.638256296587716


Q1 B - Housing dataset with Linear Ridge Reg

In [ ]:
#

Q1 C - Housing dataset with Linear Reg (Gradient Descent)

Q1 D - Housing dataset with Logistic Reg (Gradient Descent)